Formation energies
------------------

We're going to start simple here - let's run a local relaxation (optimize the unit cell and positions) using a pre-trained UMA model to compute formation energies for inorganic materials.

Note predicting formation energy using models that models trained solely on OMat24 must use OMat24 compatible references and corrections for mixing PBE and PBE+U calculations. We use MP2020-style corrections fitted to OMat24 DFT calculations. For more information see the [documentation](https://docs.materialsproject.org/methodology/materials-methodology/thermodynamic-stability/thermodynamic-stability/anion-and-gga-gga+u-mixing) at the Materials Project. The necessary references can be found using the `fairchem.data.omat` package!

````{admonition} Need to install fairchem-core or get UMA access or getting permissions/401 errors?
:class: dropdown


1. Install the necessary packages using pip, uv etc
```{code-cell} ipython3
:tags: [skip-execution]

! pip install fairchem-core fairchem-data-omat
```

2. Get access to any necessary huggingface gated models
    * Get and login to your Huggingface account
    * Request access to https://huggingface.co/facebook/UMA
    * Create a Huggingface token at https://huggingface.co/settings/tokens/ with the permission "Permissions: Read access to contents of all public gated repos you can access"
    * Add the token as an environment variable using `huggingface-cli login` or by setting the HF_TOKEN environment variable.

```{code-cell} ipython3
:tags: [skip-execution]

# Login using the huggingface-cli utility
! huggingface-cli login

# alternatively,
import os
os.environ['HF_TOKEN'] = 'MY_TOKEN'
```
````

In [1]:
from __future__ import annotations

import pprint

from ase.build import bulk
from ase.optimize import FIRE
from quacc.recipes.mlp.core import relax_job

from fairchem.core.calculate.ase_calculator import FAIRChemCalculator, FormationEnergyCalculator

# Make an Atoms object of a bulk MgO structure
atoms = bulk("MgO", "rocksalt", a=4.213)

# Run a structure relaxation
result = relax_job(
    atoms,
    method="fairchem",
    name_or_path="uma-s-1p1",
    task_name="omat",
    relax_cell=True,
    opt_params={"fmax": 1e-3, "optimizer": FIRE},
)

# Get the realxed atoms!
atoms = result["atoms"]

# Create an calculator using uma-s-1p1
calculator = FAIRChemCalculator.from_model_checkpoint("uma-s-1p1", task_name="omat")

# Now use the FormationEnergyCalculator to calculate the formation energy
# This will now return MP-style corrected formation energies
# For the omat task, this defaults to apply MP2020 style corrections with OMat24 compatibility
form_e_calc = FormationEnergyCalculator(calculator, apply_corrections=True)
atoms.calc = form_e_calc
form_energy = atoms.get_potential_energy()

INFO:httpx:HTTP Request: HEAD https://huggingface.co/facebook/UMA/resolve/main/checkpoints/uma-s-1p1.pt "HTTP/1.1 302 Found"


INFO:httpx:HTTP Request: HEAD https://huggingface.co/facebook/UMA/resolve/main/references/iso_atom_elem_refs.yaml "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: HEAD https://huggingface.co/facebook/UMA/resolve/main/references/form_elem_refs.yaml "HTTP/1.1 200 OK"


INFO:quacc.runners.prep:Calculation will run at /home/runner/work/fairchem/fairchem/docs/inorganic_materials/examples_tutorials/tmp-quacc-2025-12-13-21-22-42-555764-59941


INFO:quacc.runners.prep:Calculation results stored at /home/runner/work/fairchem/fairchem/docs/inorganic_materials/examples_tutorials/quacc-2025-12-13-21-22-42-555764-59941


INFO:httpx:HTTP Request: HEAD https://huggingface.co/facebook/UMA/resolve/main/checkpoints/uma-s-1p1.pt "HTTP/1.1 302 Found"


INFO:httpx:HTTP Request: HEAD https://huggingface.co/facebook/UMA/resolve/main/references/iso_atom_elem_refs.yaml "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: HEAD https://huggingface.co/facebook/UMA/resolve/main/references/form_elem_refs.yaml "HTTP/1.1 200 OK"


In [2]:
pprint.pprint(f"Total energy: {result['results']['energy']} eV \n Formation energy {form_energy} eV")

'Total energy: -11.866120776797066 eV \n Formation energy -6.084163336797066 eV'


Compare the results to the value of [-3.038 eV/atom reported](https://next-gen.materialsproject.org/materials/mp-1265?chemsys=Mg-O#thermodynamic_stability) in the the Materials Project!
*Note that we expect differences due to the different DFT settings used to calculate the OMat24 training data.*

Congratulations; you ran your first relaxation and predicted the formation energy of MgO using UMA and `quacc`!